In [1]:
import os
from datasets import load_dataset

In [2]:
EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-large"

In [3]:
dataset = load_dataset("dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa-qa-groq_llama3_70b_8192")

Using the latest cached version of the dataset since dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa-qa-groq_llama3_70b_8192 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/vant/.cache/huggingface/datasets/dariolopez___justicio-boe-a-1978-31229-constitucion-by-articles-qa-qa-groq_llama3_70b_8192/default/0.0.0/3a3a46039ebfc36a3554740381bea0820a84b5c2 (last modified on Mon Jun  3 16:28:25 2024).


In [4]:
dataset = dataset.rename_column('answer', 'ground_truth')
dataset = dataset.rename_column('response_groq_llama3_70b_8192', 'answer')
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'context', 'question', 'ground_truth', 'context_qa', 'answer'],
        num_rows: 515
    })
})

In [5]:
# Define la función de transformación
def extract_contexts(ejemplo):
    return {'contexts': [d['context'] for d in ejemplo['context_qa']]}

# Aplica la transformación
dataset = dataset.map(extract_contexts)
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'context', 'question', 'ground_truth', 'context_qa', 'answer', 'contexts'],
        num_rows: 515
    })
})

In [6]:
# Elimina la columna original si es necesario
dataset = dataset.remove_columns(['context_qa'])
dataset

DatasetDict({
    train: Dataset({
        features: ['number', 'context', 'question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 515
    })
})

In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key=os.environ.get('GROQ_API_KEY'), 
    # model_name="mixtral-8x7b-32768"
    model_name="llama3-70b-8192",    
)

In [9]:
from langchain_core.pydantic_v1 import BaseModel
import typing as t

class AnswerCorrectnessClassification(BaseModel):
    TP: t.List[t.Dict[str, t.Any]]
    FP: t.List[t.Dict[str, t.Any]]
    FN: t.List[t.Dict[str, t.Any]]

In [10]:
examples=[
    {
        "question": """¿Qué potencia el sol y cuál es su función principal?""",
        "answer": [
            "El sol es impulsado por la fisión nuclear, similar a los reactores nucleares en la Tierra.",
            "La función principal del sol es proporcionar luz al sistema solar.",
        ],
        "ground_truth": [
            "El sol es impulsado por la fusión nuclear, donde los átomos de hidrógeno se fusionan para formar helio.",
            "Este proceso de fusión en el núcleo del sol libera una cantidad tremenda de energía.",
            "La energía del sol proporciona calor y luz, que son esenciales para la vida en la Tierra.",
            "La luz del sol juega un papel crítico en el sistema climático de la Tierra.",
            "La luz solar ayuda a impulsar el clima y las corrientes oceánicas.",
        ],
        "classification": AnswerCorrectnessClassification.parse_obj(
            {
                "TP": [
                    {
                        "statement": "La función principal del sol es proporcionar luz al sistema solar.",
                        "reason": "Esta afirmación está algo respaldada por la verdad fundamental que menciona el sol proporcionando luz y sus roles, aunque se centra más ampliamente en la energía del sol.",
                    }
                ],
                "FP": [
                    {
                        "statement": "El sol es impulsado por la fisión nuclear, similar a los reactores nucleares en la Tierra.",
                        "reason": "Esta afirmación es incorrecta y contradice la verdad fundamental que indica que el sol es impulsado por la fusión nuclear.",
                    }
                ],
                "FN": [
                    {
                        "statement": "El sol es impulsado por la fusión nuclear, donde los átomos de hidrógeno se fusionan para formar helio.",
                        "reason": "Esta descripción precisa de la fuente de energía del sol no está incluida en la respuesta.",
                    },
                    {
                        "statement": "Este proceso de fusión en el núcleo del sol libera una cantidad tremenda de energía.",
                        "reason": "Este proceso y su importancia no se mencionan en la respuesta.",
                    },
                    {
                        "statement": "La energía del sol proporciona calor y luz, que son esenciales para la vida en la Tierra.",
                        "reason": "La respuesta solo menciona la luz, omitiendo los aspectos esenciales del calor y su necesidad para la vida, que la verdad fundamental cubre.",
                    },
                    {
                        "statement": "La luz del sol juega un papel crítico en el sistema climático de la Tierra.",
                        "reason": "Este impacto más amplio de la luz del sol en el sistema climático de la Tierra no se aborda en la respuesta.",
                    },
                    {
                        "statement": "La luz solar ayuda a impulsar el clima y las corrientes oceánicas.",
                        "reason": "El efecto de la luz solar en los patrones climáticos y las corrientes oceánicas se omite en la respuesta.",
                    },
                ],
            }
        ).dict(),
    },
    {
        "question": """¿Cuál es el punto de ebullición del agua?""",
        "answer": [
            "El punto de ebullición del agua es 100 grados Celsius al nivel del mar"
        ],
        "ground_truth": [
            "El punto de ebullición del agua es 100 grados Celsius (212 grados Fahrenheit) al nivel del mar.",
            "El punto de ebullición del agua puede cambiar con la altitud.",
        ],
        "classification": AnswerCorrectnessClassification.parse_obj(
            {
                "TP": [
                    {
                        "statement": "El punto de ebullición del agua es 100 grados Celsius al nivel del mar",
                        "reason": "Esta afirmación está directamente respaldada por la verdad fundamental que especifica el punto de ebullición del agua como 100 grados Celsius al nivel del mar.",
                    }
                ],
                "FP": [],
                "FN": [
                    {
                        "statement": "El punto de ebullición del agua puede cambiar con la altitud.",
                        "reason": "Esta información adicional sobre cómo el punto de ebullición del agua puede variar con la altitud no se menciona en la respuesta.",
                    }
                ],
            }
        ).dict(),
    },
]

In [11]:
from ragas.llms.output_parser import get_json_format_instructions

_output_instructions = get_json_format_instructions(AnswerCorrectnessClassification)

In [19]:
from ragas.llms.prompt import Prompt


CORRECTNESS_INSTRUCTIONS = """\
Dada una verdad fundamental y una respuesta, analice cada afirmación y clasifíquelas en una de las siguientes categorías:

- TP (verdadero positivo): afirmaciones presentes en la respuesta que también están respaldadas directamente por una o más afirmaciones de la verdad fundamental,
- FP (falso positivo): afirmaciones presentes en la respuesta pero no respaldadas directamente por ninguna afirmación de la verdad fundamental,
- FN (falso negativo): afirmaciones presentes en la verdad básica pero no en la respuesta.

Cada afirmación sólo puede pertenecer a una de las categorías. Razone cada clasificación.
"""
correctness_prompt = Prompt(
    name="answer_correctness",
    instruction=CORRECTNESS_INSTRUCTIONS,
    output_format_instruction=_output_instructions,
    # examples=examples,
    input_keys=["question", "answer", "ground_truth"],
    output_key="classification",
    output_type="json",
)

In [20]:
from ragas.metrics import AnswerCorrectness

answer_correctness = AnswerCorrectness(correctness_prompt=correctness_prompt)
answer_correctness

AnswerCorrectness(embeddings=None, llm=None, name='answer_correctness', evaluation_mode=<EvaluationMode.qga: 6>, correctness_prompt=Prompt(name='answer_correctness', instruction='Dada una verdad fundamental y una respuesta, analice cada afirmación y clasifíquelas en una de las siguientes categorías:\n\n- TP (verdadero positivo): afirmaciones presentes en la respuesta que también están respaldadas directamente por una o más afirmaciones de la verdad fundamental,\n- FP (falso positivo): afirmaciones presentes en la respuesta pero no respaldadas directamente por ninguna afirmación de la verdad fundamental,\n- FN (falso negativo): afirmaciones presentes en la verdad básica pero no en la respuesta.\n\nCada afirmación sólo puede pertenecer a una de las categorías. Razone cada clasificación.\n', output_format_instruction='The output should be a well-formatted JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "descrip

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={"device": "cpu"},
)

In [22]:
import time
from ragas import evaluate
from datasets import Dataset


scores=[]
for idx, row in enumerate(dataset['train']):
    time.sleep(20)  # groq limits: https://console.groq.com/settings/limits
    row = dataset['train'][idx]
    # Crear un nuevo dataset a partir de la fila seleccionada
    dataset_aux = Dataset.from_dict({key: [value] for key, value in row.items()})
    score = evaluate(dataset_aux, metrics=[answer_correctness], llm=llm, embeddings=embeddings)    
    scores.append(score['answer_correctness'])
    print(f"{idx} - {score['answer_correctness']}")
    if idx > 5:
        break

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to parse output. Returning None.
/home/vant/justicio/env3.10/lib/python3.10/site-packages/ragas/evaluation.py:276: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


0 - nan


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to parse output. Returning None.
/home/vant/justicio/env3.10/lib/python3.10/site-packages/ragas/evaluation.py:276: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


...

Failed to parse output. Returning None.
/home/vant/justicio/env3.10/lib/python3.10/site-packages/ragas/evaluation.py:276: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


5 - nan


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to parse output. Returning None.
/home/vant/justicio/env3.10/lib/python3.10/site-packages/ragas/evaluation.py:276: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


6 - nan


In [23]:
import numpy as np

np.mean(scores)

nan

In [24]:
scores

[nan, nan, nan, nan, nan, nan, nan]

In [18]:
dataset['train'][0]

{'number': 1,
 'context': 'Artículo 1\n1. España se constituye en un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico la libertad, la justicia, la igualdad y el pluralismo político.\n\n2. La soberanía nacional reside en el pueblo español, del que emanan los poderes del Estado.\n\n3. La forma política del Estado español es la Monarquía parlamentaria.',
 'question': '¿Cuáles son los valores superiores del ordenamiento jurídico en España?',
 'ground_truth': 'La libertad, la justicia, la igualdad y el pluralismo político.',
 'answer': 'Según la Constitución Española de 1978, los valores superiores del ordenamiento jurídico en España son la justicia, la libertad, la igualdad y el pluralismo político. Estos valores se establecen en el Preámbulo de la Constitución y se desarrollan a lo largo del texto constitucional.\n\nEn particular, el artículo 1.1 de la Constitución establece que "España es un Estado social y democrático de Derecho, q